# Sentiment Classification Using Deep Learning
### Submited By: Ashutosh Jatav [B16CS004]; Sanchit Taliyan [B16CS031] 

### Loading Input Data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/readme.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/sentiment labelled sentences/amazon_cells_labelled.csv
/media/ghost/New V

In [2]:
amazon_df=pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt",delimiter='\t',
                        header=None, 
                        names=['review', 'sentiment'])

imdb_df = pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt", 
                        delimiter='\t', 
                        header=None, 
                        names=['review', 'sentiment'])

yelp_df = pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt", 
                        delimiter='\t', 
                        header=None, 
                        names=['review', 'sentiment'])


In [3]:
amazon_df.head()
import pandas

In [4]:
imdb_df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
yelp_df.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


### Required Dependencies

In [6]:
import sys
from tensorflow.python.keras.layers import Layer
from tensorflow.keras import backend as K
#from keras import initializations
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional


### Attention Layer

In [7]:
def dot_product(x, kernel):

    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        super(AttentionWithContext, self).__init__(**kwargs)

        self.bias = bias

    def build(self, input_shape):
        print(input_shape)
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [8]:
data=pd.concat([amazon_df,yelp_df,imdb_df])

In [9]:
data.reset_index(drop='True',inplace=True)

In [10]:
data

,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up...,0
6,If you have several dozen or several hundred c...,0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


### Preprocessing Data 

In [11]:
import re

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")    
    

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

def additionalProcc(text):
    text = text.lower() 
    text = re.sub(r"(@[a-z]*)", "<mention>", text)
    text = re.sub(r"(&[a-z;]*)", "<none>", text)
    text = re.sub(r"(#[a-z;]*)", "<hash>", text)
    text = re.sub(r"(http|https|ftp|ftps)\:\/\/[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(\/\S*)?", "<link>", text)
    text = re.sub(r'https?://\S+', '<link>', text)
    text = re.sub(r"(www.[a-z.\/0-9]*)", "<link>", text)
    return text


In [12]:
#Extracting Reviews and Sentiments
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
data['review'] = data['review'].apply(lambda x: x.lower())

# Clean the text
data['review'] = data['review'].progress_apply(lambda x: clean_text(x))

# Clean numbers
data['review'] = data['review'].progress_apply(lambda x: clean_numbers(x))

# Clean spellings
data['review'] = data['review'].progress_apply(lambda x: replace_typical_misspell(x))
#vocab = build_vocab(data['review'])
## fill up the missing values
data['rewiew']= data['review'].fillna("_##_").values
data['review'] = data['review'].progress_apply(lambda x: additionalProcc(x))

sentences=data['review'].tolist()
label=data['sentiment'].tolist()

/media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# print some examples of sentences and labels
print("Sentences")
for i in range(10):
    #print(sentences[i],end="\n")
    print("{} {}".format(sentences[i],label[i]))
    


Sentences
so there is no way for me to plug it in here in the us unless i go by a converter .  0
good case ,  excellent value .  1
great for the jawbone .  1
tied to charger for conversations lasting more than <hash><hash> minutes . major problems !  !  0
the mic is great .  1
i have to jiggle the plug to get it to line up right to get decent volume .  0
if you have several dozen or several hundred contacts ,  then imagine the fun of sending each of them one by one .  0
if you are razr owner .  .  . you must have this !  1
needless to say ,  i wasted my money .  0
what a waste of money and time !  .  0


In [14]:
#!pip install -U "tensorflow==2.0" 
#!pip install tensorflow==1.13.1

In [15]:
#Create A Subword Datasets
import nltk
#import tensorflow_datasets as tfds
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size=10000
#tokenizer=tfds.features.text.SubwordTextEncoder.build_from_corpus(sentences,vocab_size)
#tokenizer.save_to_file('dfd')
for i in range(len(sentences)):
    sentences[i] = nltk.word_tokenize(sentences[i])
print(sentences[1])
tokenizer = Tokenizer(split=' ', char_level=False)
tokenizer.fit_on_texts(sentences)

Using TensorFlow backend.


['good', 'case', ',', 'excellent', 'value', '.']


In [16]:
print("vocab size is",vocab_size)

#check the tokenizer words
num=1
print(sentences[num])
#encoded_sentence=tokenizer.encode(sentences[num])

encoded_sentence=tokenizer.texts_to_sequences(sentences[num])
print(encoded_sentence)
print(tokenizer.word_index.items())

vocab size is 10000
['good', 'case', ',', 'excellent', 'value', '.']
[[28], [173], [], [106], [546], []]
dict_items([('.', 1), ('the', 2), (',', 3), ('and', 4), ('i', 5), ('a', 6), ('it', 7), ('is', 8), ("'", 9), ('to', 10), ('this', 11), ('of', 12), ('was', 13), ('!', 14), ('<', 15), ('>', 16), ('hash', 17), ('in', 18), ('for', 19), ('that', 20), ('not', 21), ('-', 22), ('t', 23), ('with', 24), ('s', 25), ('my', 26), ('very', 27), ('good', 28), ('you', 29), ('on', 30), ('great', 31), ('but', 32), ('have', 33), ('movie', 34), ('are', 35), ('as', 36), ('so', 37), ('phone', 38), ('film', 39), ('all', 40), ('be', 41), ('one', 42), ('0', 43), ('had', 44), ('1', 45), ('at', 46), ('food', 47), ('like', 48), ('``', 49), ('just', 50), ('place', 51), ('there', 52), ('time', 53), ('service', 54), ('they', 55), ('an', 56), ('were', 57), ('we', 58), ('if', 59), ('from', 60), ('bad', 61), ('really', 62), (')', 63), ('(', 64), ('can', 65), ('well', 66), ('out', 67), ('has', 68), ('would', 69), ('abo

In [17]:
#for i in encoded_sentence:
#    print(tokenizer.decode([i]))
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
print(sentences[num])
my_texts = list(map(sequence_to_text, encoded_sentence))
print(my_texts)

['good', 'case', ',', 'excellent', 'value', '.']
[['good'], ['case'], [], ['excellent'], ['value'], []]


In [18]:
for i,sent in enumerate(sentences):
    sentences[i]=sum(tokenizer.texts_to_sequences(sent), [])

In [19]:
#print some encoded text
for i in range(10):
    print(sentences[i],end="\n")


[37, 52, 8, 71, 130, 19, 85, 10, 390, 7, 18, 78, 18, 2, 204, 595, 5, 90, 76, 6, 2262]
[28, 173, 106, 546]
[31, 19, 2, 1105]
[2263, 10, 259, 19, 1106, 1107, 87, 88, 17, 17, 167, 1496, 321]
[2, 893, 8, 31]
[5, 33, 10, 2264, 2, 390, 10, 102, 7, 10, 299, 81, 159, 10, 102, 496, 391]
[59, 29, 33, 392, 1497, 72, 392, 2265, 1108, 160, 1109, 2, 453, 12, 894, 345, 12, 131, 42, 76, 42]
[59, 29, 35, 762, 763, 29, 260, 33, 11]
[1110, 10, 150, 5, 454, 26, 175]
[92, 6, 168, 12, 175, 4, 53]


In [20]:
import numpy as np
max_length =50
trunc_type='post'
padding_type='post'

#pad all Sequence

sequence_added=pad_sequences(sentences,maxlen=max_length,padding =padding_type,truncating=trunc_type)

In [21]:
#Separate the separate and Sentences on training and test data sets

training_size=int(len(sentences)*0.8)
train_seq=sequence_added[:training_size]
train_labels=label[:training_size]

test_seq=sequence_added[training_size:]
test_labels=label[training_size:]

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

### Loading GLove Embeddings

In [22]:
def load_glove(word_index):
    EMBEDDING_FILE = '/media/ghost/New Volume/dataset/tweet-sentiment-extraction/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (120000, 300))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        #ALLmight
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix 

In [23]:
print("Total no of Training Sequence are",len(train_seq))
print("Total no of Test Sequence are",len(test_seq))
from keras.models import Sequential
embedding_dim = 300
max_features = 5130
embedding_matrix = load_glove(tokenizer.word_index)

Total no of Training Sequence are 2198
Total no of Test Sequence are 550


/media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Model Architechture and Training

In [24]:
inp = Input(shape=(max_length,))
x = Embedding(120000,embedding_dim, weights=[embedding_matrix],input_length=max_length)(inp)
x = Bidirectional(LSTM(128,return_sequences=True))(x)
x = Bidirectional(LSTM(64,return_sequences=True))(x)
x = AttentionWithContext()(x)
x = Dense(64, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=inp, outputs=x)

(None, 50, 128)


In [25]:
#fit a model
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 300)           36000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 256)           439296    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 128)           164352    
_________________________________________________________________
attention_with_context (Atte (None, 128)               16640     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65    

In [26]:
#history=model.fit(train_seq,train_labels,epochs=50,validation_data=(test_seq,test_labels))

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
model.save_weights('sentiModel.h5')
model.load_weights('model.h5')
def plot_graphs(history, string):
        plt.plot(history.history[string])
        plt.plot(history.history['val_'+string])
        plt.xlabel("Epochs")
        plt.ylabel(string)
        plt.legend([string, 'val_'+string])
        plt.show()

#plot_graphs(history, "accuracy")
#plot_graphs(history, "loss")

### Testing [RUN THE CRAWLER NOTEBOOK BEFORE TESTING]

In [28]:
def predict_review(model, new_sentences, labels, maxlen=max_length, show_padded_sequence=True ):
    # Keep the original sentences so that we can keep using them later
    # Create an array to hold the encoded sequences
    new_sequences = []

    # Convert the new reviews to sequences
    for i, frvw in enumerate(new_sentences):
        sen = nltk.word_tokenize(frvw)
        sen = tokenizer.texts_to_sequences(sen)
        sen = sum(sen, [])
        new_sequences.append(sen)

    trunc_type='post' 
    padding_type='post'

    # Pad all sequences for the new reviews
    new_reviews_padded = pad_sequences(new_sequences, maxlen=max_length, 
                                 padding=padding_type, truncating=trunc_type)             

    classes = model.predict(new_reviews_padded)

    # The closer the class is to 1, the more positive the review is
    for x in range(len(new_sentences)):

        # We can see the padded sequence if desired
        # Print the sequence
        if (show_padded_sequence):
              print(new_reviews_padded[x])
        labels.append(classes[x])
        # Print the review as text
        #print(new_sentences[x])
        # Print its predicted class
        print(classes[x])
        print("\n")

In [ ]:
# Use the model to predict some reviews   
fake_reviews = ["I hate this phone", 
                "Everything was cold",
                "Everything was hot exactly as I wanted", 
                "Everything was green", 
                "the host seated us immediately",
                "they gave us free chocolate cake", 
                "we couldn't hear each other talk because of the shouting in the kitchen"
               ]
testLabels = []
predict_review(model, testLabels, fake_reviews)

In [ ]:
import pandas as pd
data = pd.read_csv('output.csv')
data
data['Article Content'] = data['Article Content'].apply(lambda x: x.lower())

# Clean the text
data['Article Content'] = data['Article Content'].progress_apply(lambda x: clean_text(x))

# Clean numbers
data['Article Content'] = data['Article Content'].progress_apply(lambda x: clean_numbers(x))

# Clean spellings
data['Article Content'] = data['Article Content'].progress_apply(lambda x: replace_typical_misspell(x))
#vocab = build_vocab(data['review'])
## fill up the missing values
data['Article Content']= data['Article Content'].fillna("_##_").values
data['Article Content'] = data['Article Content'].progress_apply(lambda x: additionalProcc(x))

testSentences = data['Article Content'][9].tolist()

In [ ]:
testSentences

In [ ]:
labels = []

In [ ]:
model.load_weights('sentiModel.h5')

In [ ]:
predict_review(model, testSentences, labels)

In [ ]:
labels

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 
document_sentiment = Average(labels)
print("Document Sentiment:", document_sentiment)

In [ ]:
data["labels"] = labels

In [ ]:
data

In [ ]:
data.to_csv('submit.csv')